 Создать код на Python, который загрузит на ваш локальный компьютер данные о
котировках ценных бумаг из списка SnP500 и котировки криптовалют (BTC, ETH, SOL,
XRP).

In [1]:
import os
from datetime import datetime
from itertools import chain

import pandas as pd
import yfinance

In [2]:
ROOT_FOLDER = '.'
OUTPUT_FOLDER = 'data'
REPLACE = False

sp500_tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META'] 
crypto_tickers = ['BTC-USD', 'ETH-USD', 'SOL-USD', 'XRP-USD']

start_date = '2024-01-01'
end_date = '2024-11-01'
time_frame = '1d'



In [3]:
def download_ticker(
        ticker_nm: str,
        start_date: datetime,
        end_date: datetime,
        time_frame: str
) -> pd.DataFrame:
    df = yfinance.download(
        tickers=ticker_nm,
        start=start_date,
        end=end_date,
        interval=time_frame
    )
    df = df.droplevel(1, axis=1)
    return df

In [4]:
def get_folder_nm(
        ticker_nm: str,
        time_frame: str
) -> str:
    return os.path.join(ROOT_FOLDER, OUTPUT_FOLDER, ticker_nm, time_frame)

In [5]:
def get_file_name(
        ticker_nm: str,
        start_date: datetime,
        end_date: datetime,
        time_frame: str
) -> str:
    return f'{ticker_nm}_{time_frame}_{start_date}_{end_date}.parquet'

In [6]:
def path_handler(
        ticker_nm: str,
        start_date: datetime,
        end_date: datetime,
        time_frame: str
) -> str:
    folder_path = os.path.join(ROOT_FOLDER, OUTPUT_FOLDER, ticker_nm, time_frame)
    file_nm = f'{ticker_nm}_{time_frame}_{start_date}_{end_date}.parquet'
    file_path = os.path.join(folder_path, file_nm)

    if os.path.exists(file_path):
        raise FileExistsError(f'The file has been already downloaded')
    else:
        os.makedirs(folder_path, exist_ok=True)
    return file_path

In [7]:
def save_ticker(df: pd.DataFrame, file_path: str) -> None:
    df.to_parquet(path=file_path, compression='gzip', index=False)

In [ ]:
for ticker_nm in chain(sp500_tickers, crypto_tickers):
    print(f'\n{ticker_nm}')
    
    try:
        file_path = path_handler(ticker_nm, start_date, end_date, time_frame)
    except FileExistsError as e:
        if REPLACE:
            print(f'{str(e)}.\nReplacing...')
        else:
            print(f'{str(e)}.\Skipping.')
            continue

    print(f'Downloading...')
    df = download_ticker(ticker_nm, start_date, end_date, time_frame)
    print(f'Done. {len(df)} rows')

    save_ticker(df, file_path)
    print(f'Saved to: {file_path}')


AAPL
The file has been already downloaded.
Skiping.

MSFT
The file has been already downloaded.
Skiping.

AMZN
The file has been already downloaded.
Skiping.

GOOG
The file has been already downloaded.
Skiping.

META
The file has been already downloaded.
Skiping.

BTC-USD
The file has been already downloaded.
Skiping.

ETH-USD
The file has been already downloaded.
Skiping.

SOL-USD
The file has been already downloaded.
Skiping.

XRP-USD
The file has been already downloaded.
Skiping.
